In [7]:
from database_cloudant import CloudantDB
from cloudant.design_document import DesignDocument
from cloudant.view import View
import sys



#db1 = CloudantDB("tweets_keywords")    # tweets in the specified location with keywords 
#db2 = CloudantDB("tweets_no_keywords") # tweets in the sepcified locations without keyowrds
#db3 = CloudantDB("tweets_raw")         # raw tweet data in the specified locations
db4=CloddantDB("")
#print(len(db1.get_data()))
#print(db2.get_data(n=10000))
#print(len(db3.get_data(n=10)))



    
def dataofday(date):    
    # get all data from a specific day
    db1 = CloudantDB("tweets_keywords")    # tweets in the specified location with keywords 
    db2 = CloudantDB("tweets_no_keywords") # tweets in the sepcified locations without keyowrds
    n = 100  # number of tweets need to retrieve
    ddoc = DesignDocument(db2.curDB, '_design/ddoc001')
    ddoc.fetch()
    view = View(ddoc,"view_time")
    data = []
    for row in view(group=False,limit=n,reduce=False,key=date,include_docs=True)["rows"]:
        data.append(row)
    
    db1.close()
    db2.close()
    return data

    
    

    
#db3.close()
def city_daily_popular_10words(tweetlist):
    city=[]
    for k in tweetlist:
        if k["zone"] not in city:
            city.append(k["zone"])
    alltext={}
    for c in city:
        alltext[c]=[]
    for k in tweetlist:
        for c in city:
            if k["zone"] in c:
                alltext[c].append(k["text"])
    popdict={}             
    for c in city:
        popdict[c]=top_words(alltext[c])
    return popdict


if __name__ == "main":
    ipaddress=sys.argv[-1]

    date = "2021-01-01"
    data=dataofday(date)

    citytopwords=city_daily_popular_10words(data)
    print(citytopwords)


NameError: name 'CloddantDB' is not defined

In [8]:
def get_data_summary(db="covid",viewType="month",startkey="2020-10",mode = "mean"):
    if(db=="covid"):
        db = CloudantDB('tweets_covid')
    elif(db=="vaccine"):
        db= CloudantDB("tweets_vaccine")
    elif(db=="no_keywords"):
        db=CloudantDB('tweets_no_keywords')
    else:
        print("db parameter must in [covid,vaccine,no_keywords]")
        return {}
    ddoc = DesignDocument(db.curDB,'_design/ddoc001')
    ddoc.fetch()
    if(viewType=="month"):
        view = View(ddoc, 'view_month')
    elif(viewType=="zone"):
        view = View(ddoc,"view_zone")
    elif(viewType=="zone month"):
        view=View(ddoc, 'view_zone_month')
    else:
        view = View(ddoc, 'view_time')
    result = {}
    for row in view(limit=100,reduce=True,group=True,startkey=startkey)['rows']:
        if(mode=="mean"):
            result[row['key']] = round(row['value']['sum']/row['value']['count'],6)
        elif(mode=="count"):
            result[row['key']] = row['value']['count']
    return result

def get_data(n=100,db="covid",viewType="day",startkey="2021-05-01"):
    if(db=="covid"):
        db = CloudantDB('tweets_covid')
    elif(db=="vaccine"):
        db= CloudantDB("tweets_vaccine")
    elif(db=="no_keywords"):
        db=CloudantDB('tweets_no_keywords')
    else:
        print("db parameter must in [covid,vaccine,no_keywords]")
        return []
    ddoc = DesignDocument(db.curDB,'_design/ddoc001')
    ddoc.fetch()
    if(viewType=="zone"):
        view = View(ddoc,"view_zone")
    elif(viewType=="day"):
        view = View(ddoc, 'view_time')
    else:
        return "Does not support viewType"+viewType
    result = []
    for row in view(limit=n,reduce=False,group=False,startkey=startkey,include_docs=True)['rows']:
        result.append(row)
    return result

In [18]:
monthsentidict=get_data_summary(viewType="zone")

Username: admin
Databases: ['tweets_covid', 'tweets_keywords', 'tweets_no_keywords', 'tweets_raw', 'tweets_vaccine']
Accessing db: tweets_covid


In [36]:
get_data_summary(viewType="zone month")
get_data_summary(db="vaccine",viewType="zone month")
get_data_summary(db="vaccine",viewType="zone")
get_data_summary(viewType="zone")

ConnectionError: HTTPConnectionPool(host='172.26.131.97', port=5984): Max retries exceeded with url: /_session (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000022A9954F280>: Failed to establish a new connection: [WinError 10051] 向一个无法连接的网络尝试了一个套接字操作。'))

In [19]:
monthsentidict

{'Adelaide': 0.596067,
 'Ballarat': 0.737719,
 'Brisbane': 0.567257,
 'Bunbury': 0.688183,
 'Canberra': 0.518975,
 'Geelong': 0.611571,
 'Hobart': 0.48139,
 'Melbourne': 0.589867,
 'Newcastle': 0.714606,
 'Perth': 0.56422,
 'Sydney': 0.542216}

In [15]:
get_data(db="vaccine",viewType="zone")

Username: admin
Databases: ['tweets_covid', 'tweets_keywords', 'tweets_no_keywords', 'tweets_raw', 'tweets_vaccine']
Accessing db: tweets_vaccine


[{'id': '1244942062601924610',
  'key': 'Adelaide',
  'value': 0.045734257,
  'doc': {'_id': '1244942062601924610',
   '_rev': '1-eeacfb0a05e942dd760b41d4a9b28d5b',
   'id': 1244942062601924610,
   'user_id': 1122798742942339072,
   'created_at': '2020-03-31 10:58:12',
   'text': 'Had a flu jab today and now feel like death warmed up. Whyyyyyyyyy 😭🤮',
   'place': {'type': 'city',
    'name': 'Adelaide',
    'fullname': 'Adelaide, South Australia',
    'bounding_box': '[[[138.44212992, -35.348970061], [138.780189824, -35.348970061], [138.780189824, -34.652564053], [138.44212992, -34.652564053]]]'},
   'sentiment_score': '0.045734257',
   'sentiment': 'Negative',
   'zone': 'Adelaide',
   'keywords': 'jab'}},
 {'id': '1363807123252023299',
  'key': 'Adelaide',
  'value': 0.89913225,
  'doc': {'_id': '1363807123252023299',
   '_rev': '1-909bb4182e061452cd7d3a93b86cbda0',
   'id': 1363807123252023299,
   'user_id': 3098520260,
   'created_at': '2021-02-22 11:05:31',
   'text': 'South Austr

In [34]:
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords


def top_words(textlist,top=10):
    stop_words = stopwords.words('english')
    wordfreqdict={}
    popular=[]
    tokenizer=TweetTokenizer()
    for t in textlist:
        token = tokenizer.tokenize(t)
        for w in token:
            if w not in wordfreqdict.keys():
                wordfreqdict[w]=1
            else:
                wordfreqdict[w]+=1
    sorted_values = sorted(wordfreqdict.values(), reverse=True) # Sort the values
    count=0
    for i in sorted_values:
        for k in wordfreqdict.keys():
            if wordfreqdict[k] == i:
                if len(k)>=5 and k not in popular and k not in stop_words:
                    
                    popular.append(k)
                    count+=1
                
        if count > top:
            break     
    popdict={}
    for p in popular:
        popdict[p]=wordfreqdict[p]
    return popdict
def city_daily_popular_10words():

    city=[]
    tweetlist=get_data(viewType="zone")
    for k in  tweetlist:
        if k["key"] not in city:
            city.append( k["key"])
    alltext={}
    for c in city:
        alltext[c]=[]
    for k in tweetlist:
        for c in city:
            if k["key"]  in c:
                alltext[c].append(k["doc"]["text"])
    popdict={}             
    for c in city:
        popdict[c]=top_words(alltext[c])
    return alltext

In [35]:
city_daily_popular_10words()

Username: admin
Databases: ['tweets_covid', 'tweets_keywords', 'tweets_no_keywords', 'tweets_raw', 'tweets_vaccine']
Accessing db: tweets_covid


{'Adelaide': ['‘The #COVID19 #pandemic is a stark reminder that #evidence alone is often not enough.’ Read the editorial by… https://t.co/sX6mQ9JMSF',
  'Listening to @ithinkwellHugh talk about supporting research students working remotely due to #COVID19 Tip: organise… https://t.co/GUIZwpnbrY',
  '🖤!GIG!🖤 jemmanicolemusic &amp; I will be returning from lockdown for an intimate acoustic show @graceemilyhotel !! 😀😀… https://t.co/aXj5aUizdn',
  'Covid giveth and covid taketh away',
  "Tf is going on?! Lockdown from Midnight tonight for 6 days. Hopefully we'll get to enjoy Christmas, if we get to the other side of this 😔",
  'Hmm 6 days of lockdown, might need to sort some new @lobumusic for you all!',
  'Lockdown day 2. Cant wait to go back to work tomorrow. 😂 \n@fiit_project 😂 @ Norwood, South Australia https://t.co/NTp4B9GrRy',
  'Pizza #covid @ Woodville pizza bar https://t.co/aQFCUWSfGV',
  'Watch out CAHLN.  Stroke Neurology (RAH) represent!\n\n#p.s. Santa started lifting weights  s

In [29]:
get_data(db="no_keywords",viewType="zone")

Username: admin
Databases: ['tweets_covid', 'tweets_keywords', 'tweets_no_keywords', 'tweets_raw', 'tweets_vaccine']
Accessing db: tweets_no_keywords


[{'id': '1002519101057593345',
  'key': 'Adelaide',
  'value': 0.974139,
  'doc': {'_id': '1002519101057593345',
   '_rev': '1-8a9a6c3c3630b9429653cb886fa419c3',
   'id': 1002519101057593345,
   'user_id': 3181196792,
   'created_at': '2018-06-01 11:55:54',
   'text': 'Love this !!!\nAdelaide cops 👏 https://t.co/4sRQnOft2W',
   'place': {'type': 'city',
    'name': 'Adelaide',
    'fullname': 'Adelaide, South Australia',
    'bounding_box': '[[[138.44212992, -35.348970061], [138.780189824, -35.348970061], [138.780189824, -34.652564053], [138.44212992, -34.652564053]]]'},
   'sentiment_score': '0.974139',
   'sentiment': 'Positive',
   'zone': 'Adelaide',
   'keywords': 'no keywords'}},
 {'id': '1003848262875373569',
  'key': 'Adelaide',
  'value': 0.87751937,
  'doc': {'_id': '1003848262875373569',
   '_rev': '1-ede3d1ede3e3e7b1776ab79be109acb7',
   'id': 1003848262875373569,
   'user_id': 3181196792,
   'created_at': '2018-06-05 03:57:31',
   'text': 'Wow 😟 https://t.co/gSddZeoDtA',
 